# R workshop: <font color=blue> Generating KEGG Pathway view </font>

#### MLBI@DKU

### __0. Install required R packages__

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
# BiocManager::install("biocLite")
BiocManager::install()

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'BiocVersion'

Old packages: 'httr2', 'rprojroot', 'waldo'



In [2]:
BiocManager::install("org.Hs.eg.db")
BiocManager::install("org.Mm.eg.db")
BiocManager::install("biomaRt")
BiocManager::install("gageData")
BiocManager::install("gage")
BiocManager::install("pathview")
install.packages("filesstrings")
install.packages("anndata")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'org.Hs.eg.db'

also installing the dependencies ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘XVector’, ‘GenomeInfoDb’, ‘plogr’, ‘png’, ‘Biostrings’, ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘RSQLite’, ‘S4Vectors’, ‘KEGGREST’, ‘AnnotationDbi’


Old packages: 'httr2', 'rprojroot', 'waldo'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'org.Mm.eg.db'

Old packages: 'httr2', 'rprojroot', 'waldo'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories"

In [3]:
devtools::install_github("combio-dku/KEGGPathviewGen4SCODA")
system('pip install mlbi-datasets --upgrade')
# reticulate::py_install("mlbi-datasets", method='pip')

rprojroot (2.0.4 -> 2.1.0) [CRAN]


Skipping 5 packages not available: org.Mm.eg.db, org.Hs.eg.db, gageData, gage, pathview

Installing 1 packages: rprojroot

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpYJ719C/remotes1c63427bf58/combio-dku-KEGGPathviewGen4SCODA-5969216/DESCRIPTION’ ... OK
* preparing ‘KEGGPathviewGen4SCODA’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘KEGGPathviewGen4SCODA_0.0.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



### __1. Load libraries and data__

In [4]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(filesstrings))
suppressPackageStartupMessages(library(pathview))
suppressPackageStartupMessages(library(gage))
suppressPackageStartupMessages(library(gageData))
suppressPackageStartupMessages(library(org.Hs.eg.db))
suppressPackageStartupMessages(library(org.Mm.eg.db))
suppressPackageStartupMessages(library(reticulate))
suppressPackageStartupMessages(library(anndata))
suppressPackageStartupMessages(library(KEGGPathviewGen4SCODA))

reticulate::py_require("gdown")
reticulate::py_require("mlbi-datasets")
mlbi <- reticulate::import("mlbi_datasets")

### __2A. Load your scoda-processed data__

In [ ]:
data.dir <- "./"

flst <- list.files(data.dir)
flst

[1] "BRCA_428K_GSE161529_33K_v1.h5ad"                            
[2] "sample_data"                                                
[3] "scoda_workshop_example_dataset_GSE161529_33K_results.tar.gz"

In [ ]:
file <- 'scoda_workshop_example_dataset_GSE161529_33K_results.tar.gz'
untar(file)

In [ ]:
### Load data & extract cell-gene matrix as a data.frame (rownames: cell barcode, colnames: gene symbol)
file_h5ad <- 'scoda_workshop_example_dataset_GSE161529_33K.h5ad'

adata_t <- read_h5ad(file_h5ad)
adata_t

AnnData object with n_obs × n_vars = 33785 × 22621
    obs: 'Patient', 'Description', 'Source', 'Condition', 'Menopause', 'Parity', 'Gender', 'geo_no', 'sid', 'subtype', 'subtype_detail', 'tissue', 'condition', 'sample', 'sample_rev', 'sample_ext', 'celltype_major', 'celltype_minor', 'celltype_subset', 'cnv_ref_ind', 'ploidy_score', 'ploidy_dec', 'condition_for_deg', 'sample_ext_for_deg', 'celltype_for_deg', 'celltype_for_cci', 'tumor_origin_ind'
    var: 'variable_genes', 'chr', 'spot_no'
    uns: 'CCI', 'CCI_sample', 'Celltype_marker_DB', 'DEG', 'DEG_grouping_vars', 'DEG_stat', 'DEG_vs_ref', 'DEG_vs_ref_stat', 'GSA_down', 'GSA_up', 'GSA_vs_ref_down', 'GSA_vs_ref_up', 'GSEA', 'GSEA_vs_ref', 'Pathways_DB', 'analysis_parameters', 'cnv', 'cnv_neighbors_info', 'inferploidy_summary', 'log', 'lut_sample_to_cond', 'usr_param'
    obsm: 'HiCAT_result', 'X_cnv', 'X_cnv_pca', 'X_pca', 'inferploidy_results'
    obsp: 'cnv_neighbor_graph_connectivity', 'cnv_neighbor_graph_distance'

### __2B. Or, download a preprocessed data__

In [5]:
adata_t <- mlbi$load_scoda_processed_sample_data( 'Colitis-hs' )
adata_t

AnnData object with n_obs × n_vars = 76200 × 18017
    obs: 'celltype', 'nGene', 'nUMI', 'Subject', 'Health', 'Location', 'Sample', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'celltype_org', 'sample', 'condition', 'sample_ext', 'celltype_major', 'celltype_minor', 'celltype_subset'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'variable_genes'
    uns: 'CCI', 'CCI_sample', 'Celltype_marker_DB', 'DEG', 'DEG_grouping_vars', 'DEG_stat', 'DEG_vs_ref', 'DEG_vs_ref_stat', 'GSA_down', 'GSA_up', 'GSA_vs_ref_down', 'GSA_vs_ref_up', 'GSEA', 'GSEA_vs_ref', 'HiCAT_summary', 'Pathways_DB', 'analysis_parameters', 'log', 'lut_sample_to_cond', 'usr_param'
    obsm: 'HiCAT_result', 'X_pca'

### __3. Get KEGG pathway IDs and names__

In [6]:
species <- adata_t$uns[['usr_param']][['species']]
pathways.used <- adata_t$uns[['Pathways_DB']]

df_pathways_map <- get_pathways_map( pathways.used, species, min_overlap = 0.85 )

  |======================================================================| 100%
Converting Pathways DB .. done.        


Warning message in load(url("https://pathview.uncc.edu/data/khier.rda")):
“URL 'https://pathview.uncc.edu/data/khier.rda': Timeout of 60 seconds was reached”


Error in load(url("https://pathview.uncc.edu/data/khier.rda")) : 
  cannot open the connection to 'https://pathview.uncc.edu/data/khier.rda'


### __4. Generate KEGG pathview__

In [7]:
lst.deg.all <- adata_t$uns[['DEG_vs_ref']]
lst.gsa.all <- adata_t$uns[['GSA_vs_ref_up']]

lst.fcs.all <- get_all_fold_changes( lst.deg.all, species, pval.cutoff = 0.1 )

Getting fold changes .. 
                        B cell: Healthy_vs_others(1528), Inflamed_vs_Healthy(1576), Non-inflamed_vs_Healthy(1272)
                Dendritic cell: Healthy_vs_others(1284), Inflamed_vs_Healthy(1004), Non-inflamed_vs_Healthy(1210)
              Endothelial cell: Healthy_vs_others(3303), Inflamed_vs_Healthy(3319), Non-inflamed_vs_Healthy(2516)
            Enteric glial cell: Healthy_vs_others(1842), Inflamed_vs_Healthy(2051), Non-inflamed_vs_Healthy(1256)
                    Fibroblast: Healthy_vs_others(4232), Inflamed_vs_Healthy(4157), Non-inflamed_vs_Healthy(3921)
                           ILC: Healthy_vs_others(1303), Inflamed_vs_Healthy(1130), Non-inflamed_vs_Healthy(1293)
    Intestinal Epithelial cell: Healthy_vs_others(6273), Inflamed_vs_Healthy(6930), Non-inflamed_vs_Healthy(5088)
                    Macrophage: Healthy_vs_others(3318), Inflamed_vs_Healthy(2843), Non-inflamed_vs_Healthy(3336)
                     Mast cell: Healthy_vs_others(1010), Inflam

In [8]:
target_cell <- 'Macrophage'
dir_saved <- save_kegg_pathviews( target_cell, lst.gsa.all,
                                  lst.fcs.all, df_pathways_map,
                                  species, gsa.p.val.cutoff = 1e-2 )

  |======================================================================| 100%
                    Macrophage: 3/3 - 121/121 - Fluid shear stress and atherosclerosis                     


In [ ]:
target_cell <- 'Epithelial cell'
dir_saved <- save_kegg_pathviews( target_cell, lst.gsa.all,
                                  lst.fcs.all, df_pathways_map,
                                  species, gsa.p.val.cutoff = 1e-2 )

In [ ]:
names(lst.fcs.all)

[1] "Aneuploid Epithelial cell" "B cell"                   
 [3] "Diploid Epithelial cell"   "Endothelial cell"         
 [5] "Epithelial cell"           "Fibroblast"               
 [7] "ILC"                       "Macrophage"               
 [9] "Mast cell"                 "Plasma cell"              
[11] "Smooth muscle cell"        "T cell CD4+"              
[13] "T cell CD8+"

In [ ]:
for( target_cell in names(lst.fcs.all) )
{
    dir_saved <- save_kegg_pathviews( target_cell, lst.gsa.all, lst.fcs.all, df_pathways_map, species, gsa.p.val.cutoff = 0.01 )
}

  |======================================================================| 100%
     Aneuploid Epithelial cell: 3/3 - 9/9 - Breast cancer            
  |======================================================================| 100%
                        B cell: 3/3 - 2/2 - Allograft rejection    
  |======================================================================| 100%
       Diploid Epithelial cell: 4/4 - 6/6 - Allograft rejection   
  |======================================================================| 100%
              Endothelial cell: 4/4 - 7/7 - Non-small cell lung cancer           
  |======================================================================| 100%
               Epithelial cell: 5/5 - 7/7 - Thermogenesis             
  |======================================================================| 100%
                    Fibroblast: 4/4 - 11/11 - Alzheimer disease                     
  |======================================================================| 10